# parse_data.ipynb

This notebook parses the data files used for the FP-2 assignment. 

<br>
<br>


<br>
<br>

The dependent and independent variables variables (DVs and IVs) that we are interested in are:

**DVs**:
- appliances ("Appliances" column in the CSV file)
- lights ("lights" column in the CSV file)

I combined both of them to make a new variable named total energy

**IVs**:
- temperature indoors ("T1-T9" column in the CSV file) * except for T6 as it measures outdoor
- temperature outdoors ("T_out" column in the CSV file)
- humidity indoors ("RH_1-RH_9" column in the CSV file) * except for RH_6 as it measures outdoor
- humidity outdoors ("RH_out" column in the CSV file)
- windspeed ("Windspeed" column in the CSV file)
- visibility ("Visibility" column in the CSV file)

Currently, the data is listed every 10 minutes. I aggregated the data in order for it to be shown daily. I will get the sum of the energy consumed for appliances and lights, while I will get the mean of the environmental conditions.

<br>
<br>

In [1]:
import pandas as pd

# 1. Loading the raw 10-minute dataset
df0 = pd.read_csv("energydata_complete.csv")

print(df0.head())

                  date  Appliances  lights     T1       RH_1    T2       RH_2  \
0  2016-01-11 17:00:00          60      30  19.89  47.596667  19.2  44.790000   
1  2016-01-11 17:10:00          60      30  19.89  46.693333  19.2  44.722500   
2  2016-01-11 17:20:00          50      30  19.89  46.300000  19.2  44.626667   
3  2016-01-11 17:30:00          50      40  19.89  46.066667  19.2  44.590000   
4  2016-01-11 17:40:00          60      40  19.89  46.333333  19.2  44.530000   

      T3       RH_3         T4  ...         T9   RH_9     T_out  Press_mm_hg  \
0  19.79  44.730000  19.000000  ...  17.033333  45.53  6.600000        733.5   
1  19.79  44.790000  19.000000  ...  17.066667  45.56  6.483333        733.6   
2  19.79  44.933333  18.926667  ...  17.000000  45.50  6.366667        733.7   
3  19.79  45.000000  18.890000  ...  17.000000  45.40  6.250000        733.8   
4  19.79  45.000000  18.890000  ...  17.000000  45.40  6.133333        733.9   

   RH_out  Windspeed  Visibility

In [2]:
# 2. Parsing datetime
df0['date'] = pd.to_datetime(df0['date'])
df0['day'] = df0['date'].dt.date

# 3. Creating MEAN indoor temperature & humidity (exclude T6 and RH_6 because it measures outdoor)
indoor_temp_cols = ["T1","T2","T3","T4","T5","T7","T8","T9"]
indoor_humid_cols = ["RH_1","RH_2","RH_3","RH_4","RH_5","RH_7","RH_8","RH_9"]

df0["Indoor_T"]  = df0[indoor_temp_cols].mean(axis=1)
df0["Indoor_RH"] = df0[indoor_humid_cols].mean(axis=1)

# 4. Creating Total Daily Energy
df0["Energy"] = df0["Appliances"] + df0["lights"]

# 5. Aggregating to DAILY data
daily = df0.groupby("day").agg({
    "Energy":     "sum",
    "Indoor_T":   "mean",
    "Indoor_RH":  "mean",
    "T_out":      "mean",
    "RH_out":     "mean",
    "Windspeed":  "mean",
    "Visibility": "mean"
}).reset_index()

# 6. Renaming of the final columns
daily = daily.rename(columns={
    "day":        "date",
    "Energy":     "total_energy",
    "Indoor_T":   "t_in",
    "Indoor_RH":  "h_in",
    "T_out":      "t_out",
    "RH_out":     "h_out",
    "Windspeed":  "windspeed",
    "Visibility": "visibility"
})

# 7. Removing the DATE column
df = daily.drop(columns=["date"])   

print(df.describe())

       total_energy        t_in        h_in       t_out       h_out  \
count    138.000000  138.000000  138.000000  138.000000  138.000000   
mean   14514.782609   20.812248   41.263508    7.421253   79.789718   
std     4531.328613    1.775129    3.645425    4.634690    9.033347   
min     5400.000000   16.338792   35.319880   -2.937847   50.468750   
25%    11537.500000   19.650519   38.315984    4.078646   74.563368   
50%    13920.000000   20.526286   40.547303    7.039236   80.277778   
75%    16272.500000   21.694637   44.060581    9.859809   86.014757   
max    27690.000000   25.347047   49.927046   19.372569   97.069444   

        windspeed  visibility  
count  138.000000  138.000000  
mean     4.046721   38.358297  
std      1.951432    4.661956  
min      1.000000   29.576389  
25%      2.660590   35.358507  
50%      3.637153   37.883681  
75%      4.922743   40.735491  
max     10.631944   58.600694  
